In [1]:
import pandas as pd
import numpy as np

In dieser Übung soll ein Klassifikator zur Vorhersage, ob ein Titanic-Passagier überlebt hat oder nicht, implementiert werden.

#### Einlesen des Titanic-Datensatzes

In [2]:
df = pd.read_csv('titanic.csv', usecols = ['pclass', 'survived','name','sex','age','fare'])

In [3]:
df.head()

,pclass,survived,name,sex,age,fare
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,211.3375
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,151.5500
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,151.5500
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,151.5500
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,151.5500


#### Train-Test-Split

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size=0.3, random_state=0)
X_train = X_train.copy()
X_test = X_test.copy()
y_train = X_train['survived']
y_test = X_test['survived']
X_train = X_train.drop('survived', axis=1)
X_test = X_test.drop('survived', axis=1)

#### a) Fehlende Werte

Ermitteln Sie alle Spalten des DataFrame **df**, in denen es fehlende Werte gibt.

In [5]:
df.isna().any()

pclass      False
survived    False
name        False
sex         False
age          True
fare         True
dtype: bool

Arbeiten Sie bis einschließlich Teilaufgabe g) nur auf dem Trainingsdatensatz weiter.

#### b) Feature-Engineering

Extrahieren Sie aus der Spalte **name** die Anrede des Passagiers als weiteres Feature und speichern Sie diese in einer neuen Spalte namens **title**. Verwenden Sie dazu die Methode <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html">pandas.Series.str.extract</a> und einen regulären Ausdruck. Entfernen Sie anschließend die Spalte **name** aus dem DataFrame.

In [6]:
X_train

,pclass,name,sex,age,fare
501,2,"Mellinger, Miss. Madeleine Violet",female,13.0000,19.5000
588,2,"Wells, Miss. Joan",female,4.0000,23.0000
402,2,"Duran y More, Miss. Florentina",female,30.0000,13.8583
1193,3,"Scanlan, Mr. James",male,NaN,7.7250
686,3,"Bradley, Miss. Bridget Delia",female,22.0000,7.7250
...,...,...,...,...,...
763,3,"Dean, Miss. Elizabeth Gladys ""Millvina""",female,0.1667,20.5750
835,3,"Guest, Mr. Robert",male,NaN,8.0500
1216,3,"Smyth, Miss. Julia",female,NaN,7.7333
559,2,"Sincock, Miss. Maude",female,20.0000,36.7500


In [7]:
X_train['name']

501           Mellinger, Miss. Madeleine Violet
588                           Wells, Miss. Joan
402              Duran y More, Miss. Florentina
1193                         Scanlan, Mr. James
686                Bradley, Miss. Bridget Delia
                         ...                   
763     Dean, Miss. Elizabeth Gladys "Millvina"
835                           Guest, Mr. Robert
1216                         Smyth, Miss. Julia
559                        Sincock, Miss. Maude
684               Bourke, Mrs. John (Catherine)
Name: name, Length: 916, dtype: object

In [8]:
tmp = X_train['name'].str.extract('\w*, (\w+).')

In [9]:
X_train['title'] = tmp
X_train = X_train.drop('name', axis = 1)

In [10]:
X_train

,pclass,sex,age,fare,title
501,2,female,13.0000,19.5000,Miss
588,2,female,4.0000,23.0000,Miss
402,2,female,30.0000,13.8583,Miss
1193,3,male,NaN,7.7250,Mr
686,3,female,22.0000,7.7250,Miss
...,...,...,...,...,...
763,3,female,0.1667,20.5750,Miss
835,3,male,NaN,8.0500,Mr
1216,3,female,NaN,7.7333,Miss
559,2,female,20.0000,36.7500,Miss


#### c) Datentransformation

Transformieren Sie die Spalte **sex**, indem Sie "male" durch "0" und "female" durch "1" ersetzen.

In [11]:
X_train = X_train.replace(['female', 'male'], [1,0])

#### d) Imputation

Ersetzen Sie unter Verwendung der Klasse <a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html">SimpleImputer</a> fehlende Werte in den Spalten **age** und **fare** durch die jeweiligen Spaltenmittelwerte. 

In [12]:
from sklearn.impute import SimpleImputer

imp_mean_age = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean_age = imp_mean_age.fit(X_train[['age']])
X_train['age'] = imp_mean_age.transform(X_train[['age']])

imp_mean_fare = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean_fare = imp_mean_fare.fit(X_train[['fare']])
X_train['fare'] = imp_mean_fare.transform(X_train[['fare']])

X_train

,pclass,sex,age,fare,title
501,2,1,13.000000,19.5000,Miss
588,2,1,4.000000,23.0000,Miss
402,2,1,30.000000,13.8583,Miss
1193,3,0,30.231444,7.7250,Mr
686,3,1,22.000000,7.7250,Miss
...,...,...,...,...,...
763,3,1,0.166700,20.5750,Miss
835,3,0,30.231444,8.0500,Mr
1216,3,1,30.231444,7.7333,Miss
559,2,1,20.000000,36.7500,Miss


#### e) One Hot Encoding

Transformieren Sie die Spalte **title**, indem Sie alle Werte, die nicht zu
den vier häufigsten Ausprägungen gehören, durch "other" ersetzen. Erzeugen Sie anschließend
durch One Hot Encoding für jede Ausprägung von **title** eine neue Spalte, in der binär kodiert ist,
ob bei einem Passagier die jeweilige Ausprägung vorliegt oder nicht. Entfernen Sie anschließend die Spalte **title**.

Hinweise: pandas.where, pandas.get_dummies

In [13]:
X_train['title'].value_counts()[:10].index.tolist()

['Mr', 'Miss', 'Mrs', 'Master', 'Dr', 'Rev', 'Col', 'Mlle', 'Jonkheer', 'Ms']

In [14]:
top4titles = X_train['title'].value_counts()[:4].index.tolist()

In [15]:
X_train['title'].where(X_train['title'].isin(top4titles), 'other', inplace = True)

In [16]:
X_train

,pclass,sex,age,fare,title
501,2,1,13.000000,19.5000,Miss
588,2,1,4.000000,23.0000,Miss
402,2,1,30.000000,13.8583,Miss
1193,3,0,30.231444,7.7250,Mr
686,3,1,22.000000,7.7250,Miss
...,...,...,...,...,...
763,3,1,0.166700,20.5750,Miss
835,3,0,30.231444,8.0500,Mr
1216,3,1,30.231444,7.7333,Miss
559,2,1,20.000000,36.7500,Miss


In [17]:
X_train = pd.get_dummies(X_train)

#### f) Modelltraining

Trainieren Sie auf dem Trainingsdatensatz ein logistisches Regressionsmodell.

In [18]:
X_train

,pclass,sex,age,fare,title_Master,title_Miss,title_Mr,title_Mrs,title_other
501,2,1,13.000000,19.5000,0,1,0,0,0
588,2,1,4.000000,23.0000,0,1,0,0,0
402,2,1,30.000000,13.8583,0,1,0,0,0
1193,3,0,30.231444,7.7250,0,0,1,0,0
686,3,1,22.000000,7.7250,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
763,3,1,0.166700,20.5750,0,1,0,0,0
835,3,0,30.231444,8.0500,0,0,1,0,0
1216,3,1,30.231444,7.7333,0,1,0,0,0
559,2,1,20.000000,36.7500,0,1,0,0,0


In [19]:
from sklearn.linear_model import LogisticRegression


clf = LogisticRegression(random_state=0, max_iter = 2000).fit(X_train, y_train)

#### g) Güte des Modells auf dem Trainingsdatensatz

Bestimmen Sie die Accuracy und den ROC-AUC-Score auf dem Trainingsdatensatz.

In [20]:
from sklearn.metrics import confusion_matrix

def stats(true, pred):
    c = confusion_matrix(true, pred)
    print("Confusion Matrix: \n", c)

    TP = c[0][0]
    FP = c[0][1]
    FN = c[1][0]
    TN = c[1][1]

    print("TN:", TN, "TP:",TP,"FN:",FN,"FP:",FP)

    acc = (TP+TN)/(TP+TN+FP+FN)

    print("Accuracy: \n", acc)

    recall = TP/(TP+FN)

    print("Recall: \n",recall)

    precision = TP/(TP+FP)

    print("Precision: \n",precision)

In [21]:
from sklearn import metrics


#ROC_AUC_score_test = metrics.roc_auc_score(y_test, clf.decision_function(X_test))
ROC_AUC_score_train = metrics.roc_auc_score(y_train, clf.decision_function(X_train))

print("ROC_AUC_scores: \nTrain: ", ROC_AUC_score_train)

stats(y_train, clf.predict(X_train))

ROC_AUC_scores: 
Train:  0.8439762703847761
Confusion Matrix: 
 [[471  92]
 [ 99 254]]
TN: 254 TP: 471 FN: 99 FP: 92
Accuracy: 
 0.7914847161572053
Recall: 
 0.8263157894736842
Precision: 
 0.8365896980461812


#### h) Modellanwendung auf dem Testdatensatz

Wenden Sie nun das Modell auf den Testdatensatz an. Führen Sie dazu dieselben Datentransformationsschritte wie beim Trainingsdatensatz aus. Achten Sie darauf, dass Sie nur auf Information des Trainingsdatensatzes zurückgreifen (z.B. die Mittelwerte beim Imputieren oder die vier häufigsten Ausprägungen der Spalte **title**). Bestimmen Sie Accuracy und ROC-AUC-Score auf dem Testdatensatz und bewerten Sie das resultierende Modell.

In [22]:
#Titel
tmp = X_test['name'].str.extract('\w*, (\w+).')
X_test['title'] = tmp
X_test = X_test.drop('name', axis = 1)

In [23]:
#Binary Gender
X_test = X_test.replace(['female', 'male'], [1,0])

In [24]:
#Inputieren
X_test['age'] = imp_mean_age.transform(X_test[['age']])
X_test['fare'] = imp_mean_fare.transform(X_test[['fare']])

In [25]:
#Title other
X_test['title'].where(X_test['title'].isin(top4titles), 'other', inplace = True)

In [26]:
#one hot
X_test = pd.get_dummies(X_test)

In [27]:
#Predict
ROC_AUC_score_test = metrics.roc_auc_score(y_test, clf.decision_function(X_test))
print("ROC_AUC_scores: \nTest: ", ROC_AUC_score_test)
stats(y_test, clf.predict(X_test))

ROC_AUC_scores: 
Test:  0.8444776284497538
Confusion Matrix: 
 [[208  38]
 [ 46 101]]
TN: 101 TP: 208 FN: 46 FP: 38
Accuracy: 
 0.7862595419847328
Recall: 
 0.8188976377952756
Precision: 
 0.8455284552845529


## Aufgabe 2

In dieser Aufgabe soll der Datentransformationsprozess und die Klassifikation mit Hilfe einer <a href="https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html">Pipeline</a> realisiert werden.

#### Daten erneut einlesen

Hier wird der Datensatz erneut eingelesen und diesmal nur die Spalten **survived**, **pclass**, **sex**, **age** und **fare** beibehalten. Anschließend erfolgt der Train-Test-Split und das Abspalten der Zielvariable.

In [44]:
df = pd.read_csv('titanic.csv', usecols = ['pclass', 'survived','sex','age','fare'])
df['pclass'] = df['pclass'].astype(str)
X_train, X_test = train_test_split(df, test_size=0.3, random_state=0)
X_train = X_train.copy()
X_test = X_test.copy()
y_train = X_train['survived']
y_test = X_test['survived']
X_train = X_train.drop('survived', axis=1)
X_test = X_test.drop('survived', axis=1)

In [45]:
X_train.head()

,pclass,sex,age,fare
501,2,female,13.0,19.5000
588,2,female,4.0,23.0000
402,2,female,30.0,13.8583
1193,3,male,NaN,7.7250
686,3,female,22.0,7.7250


#### a) Eigenen Imputer schreiben

Implementieren Sie einen eigenen Imputer namens **OutlierImputer**. Dieser soll auf numerische Spalten angewendet werden und fehlende Werte in einer Spalte durch 
$x_{min}-(x_{max}-x_{min})$ ersetzen, wobei $x_{max}$ der maximale und $x_{min}$ der minimale Wert der Spalte des Trainingsdatensatzes ist. Leiten Sie Ihren Imputer von den Klassen TransformerMixin und BaseEstimator ab.

from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator

class OutlierImputer(BaseEstimator, TransformerMixin):
    
    def fit(self, column):
        xmin = column.min()
        xmax = column.max()
        self.xval = xmin-(xmax-xmin)
    
    def transform(self, column):
        return column.fillna(self.xval)
        
        

In [50]:
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from pandas.api.types import is_numeric_dtype


class OutlierImputer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.xval = {}
    
    def fit(self, df,y=None):
        for c in df.columns:
            if(not(is_numeric_dtype(df[c].dtype))):
                continue
            xmin = df[c].min()
            xmax = df[c].max()
            self.xval[c] = xmin-(xmax-xmin)
        return self
    
    def transform(self, df,y=None):
        df_t = df.copy()
        for c in df.columns:
            if(not(is_numeric_dtype(df[c].dtype))):
                continue
            df_t[c] = df_t[c].fillna(self.xval[c])
        return df_t
        
        

x = OutlierImputer()
x.fit(X_train)
x.transform(X_train)

#### b) ColumnTransformer und Pipeline

Mit Hilfe eines <a href="https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html">ColumnTransformers</a> können bestimmte Transformer auf Teilmengen der Datensatz-Spalten angewendet werden (z.B. die numerischen und die kategorischen).

Legen Sie einen ColumnTransformer an, der auf die numerischen Spalten den OutlierImputer aus Teilaufgabe a) und den <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html">StandardScaler</a> anwendet, und auf die kategorischen Spalten den <a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html">SimpleImputer</a> (mit strategy='constant') und einen <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html">OneHotEncoder</a>. Legen Sie anschließend eine Pipeline an, die aus dem ColumnTransformer und einem LogisticRegression Estimator besteht. Fitten Sie die Pipeline auf dem Trainingsdatensatz und berechnen Sie Accuracy und ROC-AUC-Score auf dem Trainingsdatensatz

In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from pandas.api.types import is_numeric_dtype


numeric_features = ["age", "fare", "pclass"]
numeric_transformer = Pipeline(
    steps=[("imputer", OutlierImputer()), ("scaler", StandardScaler())]
)

categorical_features = ["sex"]
categorical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy='constant')), ("OneHotEncoder", OneHotEncoder())]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.776


#### c) Modellanwendung auf dem Testdatensatz

Wenden Sie die Pipeline auf den Testdatensatz an. Berechnen Sie den ROC AUC-Score und die Accuracy für den Testdatensatz.

In [60]:
y_pred = clf.predict(X_test)

acc = metrics.accuracy_score(y_test, y_pred)
roc_auc = metrics.roc_auc_score(y_test, clf.decision_function(X_test))

print("Acc: ", acc, " ROC_AUC: ", roc_auc)

Acc:  0.7760814249363868  ROC_AUC:  0.8011310215142966
